In [ ]:
from google.colab import drive
drive.mount('drive')
drive = 'drive/My Drive/Spring 2021/Stringer/Data/'
# drive = '/content/drive/MyDrive/Stringer/Data/'

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
# read in cluster output (underlying tweets and top n keywords) for current and prior pulls
cluster1_tweets = pd.read_csv(drive + 'output/' + 'tweets_cluster_output2021-04-30-16-11-00-RHI_15000_2021-04-22_search.csv', index_col=0)
cluster1_keywords = pd.read_csv(drive + 'output/' + 'tweets_cluster_keywords2021-04-30-16-11-00-RHI_15000_2021-04-22_search.csv', index_col=0)
cluster2_tweets = pd.read_csv(drive + 'output/' + 'tweets_cluster_output2021-05-01-15-41-07-KE_15000_2021-04-29_search.csv', index_col=0)
cluster2_keywords = pd.read_csv(drive + 'output/' + 'tweets_cluster_keywords2021-05-01-15-41-07-KE_15000_2021-04-29_search.csv', index_col=0)
cluster1_tweets.head(2)

,tweet_id,user,created_on,tweet_text,fav_count,retweet_count,user_description,user_verified,user_follower_count,is_retweet,hashtags,is_quote,quote_text,tweetsProc,tweets_token,clusters
0,1385382910925045763,WorriedTeacher3,2021-04-22 23:59:59,@fitz_meagan It’s sad that they will probably ...,0,0,Reactive approaches,False,55,1,[],0,None,sad probably included covid death total,"{'probably', 'death', 'covid', 'included', 'sa...",6
1,1385382910467870725,KIN20200604,2021-04-22 23:59:59,The Institute of Military Veterinary Medicine ...,0,1226,20200604新中国联邦人！永不为奴！一切已经开始🙏🙏🙏！,False,3234,1,[],0,None,institute military veterinary medicine changch...,"{'changchun', 'experiment', 'conducted', 'mili...",0


In [ ]:
cluster1_keywords.head(30)

,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7
0,vaccination,new,got,johnson,india,vaccine,covid
1,crime,case,second,boris,flight,appointment,hospital
2,hate,apr,pfizer,ron,pakistan,available,patient
3,coronavirus,death,moderna,dyson,case,dos,oxygen
4,senate,access,shot,tory,canada,getting,bed
5,jab,reported,dose,text,ban,clinic,like
6,government,coronavirus,getting,tax,record,rollout,positive
7,read,report,arm,super,variant,vaccinated,delhi
8,health,york,vaccine,climate,trudeau,people,test
9,american,walgreens,feeling,prime,day,passport,time


In [ ]:
cluster2_keywords.head(30)

,cluster1,cluster2,cluster3,cluster4,cluster5,cluster6,cluster7
0,vaccine,pfizer,new,johnson,need,got,covid
1,coronavirus,shot,case,boris,like,shot,india
2,vaccination,dose,active,alize,help,second,year
3,health,second,apr,tory,look,moderna,died
4,appointment,dos,store,dwayne,donate,dose,relief
5,vaccinated,moderna,death,prime,oxygen,vaccine,hospital
6,jab,got,walgreen,flat,feel,hour,death
7,myturn,sore,access,think,people,arm,patient
8,selecting,begin,record,john,country,feel,modi
9,eligible,arm,york,game,india,week,going


In [ ]:
allClust1 = []
for x in range(len(cluster1_keywords.columns)):
  allClust1.append(cluster1_keywords.iloc[:, x].tolist())
# allClust

allClust2 = []
for x in range(len(cluster2_keywords.columns)):
  allClust2.append(cluster2_keywords.iloc[:, x].tolist())

In [ ]:
results = {}
closest = []
for two, cluster2 in enumerate(allClust2):
  # print('cluster2:', two )
  options = []
  for one, cluster1 in enumerate(allClust1):
    # print('cluster1:', one )
    a = set(cluster2)
    b = set(cluster1)
    intersection = a.intersection(b)
    universe = a | b
    symmetric_diff = a.symmetric_difference(b)
    # print(a)
    # print(b)
    # print(symmetric_diff)

# compare clusters
    sameof10 = float(len(intersection)) / len(a) * 100 # what percentage in common
    sameofTotal = float(len(intersection)) / len(universe) * 100 # what percentage of total number of words were in common
    differenceof10 = float(len(symmetric_diff)) / len(a) * 100 # what percentage is different
    # print(sameofTotal, sameof10)
    options.append([one, differenceof10, sameofTotal, sameof10])
  # print(options)  

# find the best option cluster
  bestOption = max(options, key=lambda x: x[-1])
  closest.append(allClust1[bestOption[0]])
  # print('Original', allClust2[two], '\nBest Option', allClust1[bestOption[0]], '\n', bestOption, '\n\n')
  results[two] = {}
  results[two]['closeClust'] = bestOption[0]
  results[two]['differenceof10'] = bestOption[1]
  results[two]['sameofTotal'] = bestOption[2]
  results[two]['sameof10'] = bestOption[3]

In [ ]:
# check clusters calculations for best option
for key, value in results.items():
  # print('New Cluster', key, ':', value)
  print('New Cluster', key, ':', allClust2[key]) 
  print('Old Cluster', results[key]['closeClust'], ':', allClust1[results[key]['closeClust']], '\n%common in top 10', results[key]['sameof10'], ' -  %different in top 10 words', results[key]['differenceof10'], ' -  %total common words', results[key]['sameofTotal'], '\n\n')

New Cluster 0 : ['vaccine', 'coronavirus', 'vaccination', 'health', 'appointment', 'vaccinated', 'jab', 'myturn', 'selecting', 'eligible']
Old Cluster 0 : ['vaccination', 'crime', 'hate', 'coronavirus', 'senate', 'jab', 'government', 'read', 'health', 'american'] 
%common in top 10 40.0  -  %different in top 10 words 120.0  -  %total common words 25.0 


New Cluster 1 : ['pfizer', 'shot', 'dose', 'second', 'dos', 'moderna', 'got', 'sore', 'begin', 'arm']
Old Cluster 2 : ['got', 'second', 'pfizer', 'moderna', 'shot', 'dose', 'getting', 'arm', 'vaccine', 'feeling'] 
%common in top 10 70.0  -  %different in top 10 words 60.0  -  %total common words 53.84615384615385 


New Cluster 2 : ['new', 'case', 'active', 'apr', 'store', 'death', 'walgreen', 'access', 'record', 'york']
Old Cluster 1 : ['new', 'case', 'apr', 'death', 'access', 'reported', 'coronavirus', 'report', 'york', 'walgreens'] 
%common in top 10 60.0  -  %different in top 10 words 80.0  -  %total common words 42.857142857142854

In [ ]:
# get evolved, new, and depreciated clusters
# for c in closest:
  # print(c)
evolved = []
new = []
depreciated = []
for key, value in results.items():
# threshold of 50% or more common keywords
  if results[key]['sameof10'] > 50.0:
    evolved.append(key)
    # print('Evolved Cluster:', key, ':', allClust2[key])
    # print('   Old Cluster', results[key]['closeClust'], ':', allClust1[results[key]['closeClust']], '\n%common in top 10', results[key]['sameof10'], ' -  %different in top 10 words', results[key]['differenceof10'], ' -  %total common words', results[key]['sameofTotal'], '\n\n')
  else:
    new.append(key)
    # print(results[key]['closeClust'])
    # print(allClust1[results[key]['closeClust']])
    x = allClust1[results[key]['closeClust']]
    closest.remove(x)
    # print('New Cluster:', allClust2[key], '\n\n')

for i, old in enumerate(allClust1):
  if old not in closest:
    depreciated.append(i)
    # print('Depreciated Cluster:', old, '\n\n')
# print(depreciated)

In [ ]:
print('new - cluster 2', new, '\nevolved - cluster 2', evolved, '\ndepreciated - cluster 1', depreciated)

newClustDat = cluster2_tweets[cluster2_tweets['clusters'].isin(new)]
evolvedClustDat = cluster2_tweets[cluster2_tweets['clusters'].isin(evolved)]
depreciatedClustDat = cluster1_tweets[cluster1_tweets['clusters'].isin(depreciated)]
evolvedClustDat_old = cluster1_tweets[~cluster1_tweets['clusters'].isin(depreciated)]

new - cluster 2 [0, 3, 4, 6] 
evolved - cluster 2 [1, 2, 5] 
depreciated - cluster 1 [0, 3, 4, 5, 6]


In [ ]:
# get velocity stats
def getStats(df):
  stats = df.groupby('clusters', as_index=False).agg({'tweet_id': 'nunique', 'user': 'nunique', 'fav_count' : 'sum',
                     'user_follower_count' : 'sum', 'user_verified': 'sum'})
  stats.rename(columns={'tweet_id': 'num_tweets', 'user': 'num_user', 'user_follower_count': 'num_followers', 'user_verified': 'num_verified'}, inplace=True)
  stats['avg_tweets'] = round(stats['num_tweets']/7, 0)
  return stats
oldEvStats = getStats(evolvedClustDat_old)
oldEvStats

,clusters,num_tweets,num_user,fav_count,num_followers,num_verified,avg_tweets
0,1,418,384,462,8891627,14,60.0
1,2,499,491,2257,1182221,7,71.0


In [ ]:
newEvStats = getStats(evolvedClustDat)
newEvStats

,clusters,num_tweets,num_user,fav_count,num_followers,num_verified,avg_tweets
0,1,250,235,422,7897001,4,36.0
1,2,571,485,481,12152933,26,82.0
2,5,398,395,1333,739661,6,57.0


In [ ]:
newStats = getStats(newClustDat)
newStats

,clusters,num_tweets,num_user,fav_count,num_followers,num_verified,avg_tweets
0,0,3894,3386,4109,57214267,126,556.0
1,3,359,327,305,1159155,7,51.0
2,4,845,804,1020,2911852,9,121.0
3,6,1455,1360,2070,3136939,21,208.0


In [ ]:
print('------------------------------------------------------')
for d in depreciated:
  print('Depreciated Cluster:', d, ':', allClust1[d], '\n\n')
print('------------------------------------------------------')
for n in new:
  print('New Cluster:', n, ':', allClust2[n], '\n\n')
  sNew = newStats[newStats['clusters'] == n].T
  sNew = sNew.iloc[1:]
  sNew['idx'] = range(len(sNew))
  sNew = sNew.reset_index().set_index('idx')
  sNew.rename(columns={list(sNew)[0]:'stats', list(sNew)[1]:'value'}, inplace=True)
  sNew = sNew.set_index('stats')
  print(sNew, '\n\n')
print('------------------------------------------------------')
for e in evolved:
  print('Evolved Cluster:', e, ':', allClust2[e])
  print('Old Cluster', results[e]['closeClust'], ':', allClust1[results[e]['closeClust']], '\n\n%common in top 10', results[e]['sameof10'], ' -  %different in top 10 words', results[e]['differenceof10'], ' -  %total common words', results[e]['sameofTotal'])
  cNew = newEvStats[newEvStats['clusters'] == e]
  cNew = cNew.T
  cNew['idx'] = range(len(cNew))
  cNew = cNew.reset_index().set_index('idx')
  cNew.rename(columns={list(cNew)[0]:'stats', list(cNew)[1]:'value'}, inplace=True)
  cOld = oldEvStats[oldEvStats['clusters'] == results[e]['closeClust']]
  cOld = cOld.T
  cOld['idx'] = range(len(cOld))
  cOld = cOld.reset_index().set_index('idx')
  cOld.rename(columns={list(cOld)[0]:'stats', list(cOld)[1]:'value'}, inplace=True)
  # print('new', cNew, '\nolf', cOld, '\n\n')
  df = cNew.set_index('stats').div(cOld.set_index('stats')).sub(1).mul(100).reset_index()
  df = df.iloc[1:]
  print('Stats % change:\n', df, '\n\n')

------------------------------------------------------
Depreciated Cluster: 0 : ['vaccination', 'crime', 'hate', 'coronavirus', 'senate', 'jab', 'government', 'read', 'health', 'american'] 


Depreciated Cluster: 3 : ['johnson', 'boris', 'ron', 'dyson', 'tory', 'text', 'tax', 'super', 'climate', 'prime'] 


Depreciated Cluster: 4 : ['india', 'flight', 'pakistan', 'case', 'canada', 'ban', 'record', 'variant', 'trudeau', 'day'] 


Depreciated Cluster: 5 : ['vaccine', 'appointment', 'available', 'dos', 'getting', 'clinic', 'rollout', 'vaccinated', 'people', 'passport'] 


Depreciated Cluster: 6 : ['covid', 'hospital', 'patient', 'oxygen', 'bed', 'like', 'positive', 'delhi', 'test', 'time'] 


------------------------------------------------------
New Cluster: 0 : ['vaccine', 'coronavirus', 'vaccination', 'health', 'appointment', 'vaccinated', 'jab', 'myturn', 'selecting', 'eligible'] 


                    value
stats                    
num_tweets         3894.0
num_user           3386.0

In [ ]:
# create cluster assignment output
rows = []
for d in depreciated:
  rows.append(['Old Cluster #' + str(d), 'Depreciated', allClust1[d]])
  # print('Depreciated Cluster:', d, ':', allClust1[d], '\n\n')
for n in new:
  rows.append(['New Cluster #' + str(n), 'New', allClust2[n]])
for e in evolved:
  rows.append(['New Cluster #' + str(e), 'Evolved', allClust2[e]])
  rows.append(['Old Cluster #' + str(results[e]['closeClust']), 'Evolved',allClust1[results[e]['closeClust']]])
clusterMap = pd.DataFrame(rows, columns=['Cluster', 'Status', 'Top Words'])
# print(clusterMap)
clusterMap.to_csv(drive + 'output/' + 'clusterMap.csv', index=False)
clusterMap.head(15)

,Cluster,Status,Top Words
0,Old Cluster #0,Depreciated,"[vaccination, crime, hate, coronavirus, senate..."
1,Old Cluster #3,Depreciated,"[johnson, boris, ron, dyson, tory, text, tax, ..."
2,Old Cluster #4,Depreciated,"[india, flight, pakistan, case, canada, ban, r..."
3,Old Cluster #5,Depreciated,"[vaccine, appointment, available, dos, getting..."
4,Old Cluster #6,Depreciated,"[covid, hospital, patient, oxygen, bed, like, ..."
5,New Cluster #0,New,"[vaccine, coronavirus, vaccination, health, ap..."
6,New Cluster #3,New,"[johnson, boris, alize, tory, dwayne, prime, f..."
7,New Cluster #4,New,"[need, like, help, look, donate, oxygen, feel,..."
8,New Cluster #6,New,"[covid, india, year, died, relief, hospital, d..."
9,New Cluster #1,Evolved,"[pfizer, shot, dose, second, dos, moderna, got..."


In [ ]:
test = pd.read_csv(drive + 'output/' + 'clusterMap.csv')
test.head(15)

,Cluster,Status,Top Words
0,Old Cluster #0,Depreciated,"['vaccination', 'crime', 'hate', 'coronavirus'..."
1,Old Cluster #3,Depreciated,"['johnson', 'boris', 'ron', 'dyson', 'tory', '..."
2,Old Cluster #4,Depreciated,"['india', 'flight', 'pakistan', 'case', 'canad..."
3,Old Cluster #5,Depreciated,"['vaccine', 'appointment', 'available', 'dos',..."
4,Old Cluster #6,Depreciated,"['covid', 'hospital', 'patient', 'oxygen', 'be..."
5,New Cluster #0,New,"['vaccine', 'coronavirus', 'vaccination', 'hea..."
6,New Cluster #3,New,"['johnson', 'boris', 'alize', 'tory', 'dwayne'..."
7,New Cluster #4,New,"['need', 'like', 'help', 'look', 'donate', 'ox..."
8,New Cluster #6,New,"['covid', 'india', 'year', 'died', 'relief', '..."
9,New Cluster #1,Evolved,"['pfizer', 'shot', 'dose', 'second', 'dos', 'm..."


In [ ]:
# create cluster stats outputs
newDF = pd.DataFrame(columns = ['stats'])
evolvedDF = pd.DataFrame(columns = ['stats'])
for n in new:
  sNew = newStats[newStats['clusters'] == n].T
  sNew = sNew.iloc[1:]
  sNew['idx'] = range(len(sNew))
  sNew = sNew.reset_index().set_index('idx')
  sNew.rename(columns={list(sNew)[0]:'stats', list(sNew)[1]:'value'}, inplace=True)
  sNew = sNew.append({'stats':'New Cluster Words', 'value': allClust2[n]}, ignore_index=True) #-----------
  print(allClust2[n], '\n')
  sNew = sNew.set_index('stats')
  sNew.rename(columns={'value': 'New Cluster #' + str(n)}, inplace=True)
  newDF = pd.merge(newDF, sNew, how = 'outer', on = 'stats')
newDF = newDF.transpose().reset_index().rename(columns={'index':'stats'})
newDF.columns.name = None
newDF.columns = newDF.iloc[0]
newDF = newDF.iloc[1:]

for e in evolved:
  cNew = newEvStats[newEvStats['clusters'] == e]
  cNew = cNew.T
  cNew['idx'] = range(len(cNew))
  cNew = cNew.reset_index().set_index('idx')
  cNew.rename(columns={list(cNew)[0]:'stats', list(cNew)[1]:'value'}, inplace=True)
  cOld = oldEvStats[oldEvStats['clusters'] == results[e]['closeClust']]
  cOld = cOld.T
  cOld['idx'] = range(len(cOld))
  cOld = cOld.reset_index().set_index('idx')
  cOld.rename(columns={list(cOld)[0]:'stats', list(cOld)[1]:'value'}, inplace=True)
  df = cNew.set_index('stats').div(cOld.set_index('stats')).sub(1).mul(100).reset_index()
  df = df.iloc[1:]
  df = df.round({'value': 1})
  df = df.append({'stats':'Old Cluster #', 'value':cOld.loc[0,'value']}, ignore_index=True)
  df = df.append({'stats':'Old Cluster Words', 'value': allClust1[results[e]['closeClust']]}, ignore_index=True)
  df = df.append({'stats':'New Cluster Words', 'value': allClust2[e]}, ignore_index=True)
  df.rename(columns={'value': 'New Cluster #' + str(e)}, inplace=True)
  evolvedDF = pd.merge(evolvedDF, df, how = 'outer', on = 'stats')
evolvedDF = evolvedDF.transpose().reset_index().rename(columns={'index':'stats'})
evolvedDF.columns.name = None
evolvedDF.columns = evolvedDF.iloc[0]
evolvedDF = evolvedDF.iloc[1:]
newDF.to_csv(drive + 'output/' + 'newStats.csv', index=False)
evolvedDF.to_csv(drive + 'output/' + 'evolvedPercentChange.csv', index=False)
newDF

['vaccine', 'coronavirus', 'vaccination', 'health', 'appointment', 'vaccinated', 'jab', 'myturn', 'selecting', 'eligible'] 

['johnson', 'boris', 'alize', 'tory', 'dwayne', 'prime', 'flat', 'think', 'john', 'game'] 

['need', 'like', 'help', 'look', 'donate', 'oxygen', 'feel', 'people', 'country', 'india'] 

['covid', 'india', 'year', 'died', 'relief', 'hospital', 'death', 'patient', 'modi', 'going'] 



,stats,num_tweets,num_user,fav_count,num_followers,num_verified,avg_tweets,New Cluster Words
1,New Cluster #0,3894,3386,4109,5.72143e+07,126,556,"[vaccine, coronavirus, vaccination, health, ap..."
2,New Cluster #3,359,327,305,1.15916e+06,7,51,"[johnson, boris, alize, tory, dwayne, prime, f..."
3,New Cluster #4,845,804,1020,2.91185e+06,9,121,"[need, like, help, look, donate, oxygen, feel,..."
4,New Cluster #6,1455,1360,2070,3.13694e+06,21,208,"[covid, india, year, died, relief, hospital, d..."


In [ ]:
test = pd.read_csv(drive + 'output/' + 'newStats.csv')
# test = pd.read_csv(drive + 'output/' + 'evolvedPercentChange.csv')
test.head(15)

,stats,num_tweets,num_user,fav_count,num_followers,num_verified,avg_tweets,New Cluster Words
0,New Cluster #0,3894.0,3386.0,4109.0,57214267.0,126.0,556.0,"['vaccine', 'coronavirus', 'vaccination', 'hea..."
1,New Cluster #3,359.0,327.0,305.0,1159155.0,7.0,51.0,"['johnson', 'boris', 'alize', 'tory', 'dwayne'..."
2,New Cluster #4,845.0,804.0,1020.0,2911852.0,9.0,121.0,"['need', 'like', 'help', 'look', 'donate', 'ox..."
3,New Cluster #6,1455.0,1360.0,2070.0,3136939.0,21.0,208.0,"['covid', 'india', 'year', 'died', 'relief', '..."


In [ ]:
test = pd.read_csv(drive + 'output/' + 'evolvedPercentChange.csv')
test.head(15)

,stats,num_tweets,num_user,fav_count,num_followers,num_verified,avg_tweets,Old Cluster #,Old Cluster Words,New Cluster Words
0,New Cluster #1,-49.9,-52.1,-81.3,568.0,-42.9,-49.3,2.0,"['got', 'second', 'pfizer', 'moderna', 'shot',...","['pfizer', 'shot', 'dose', 'second', 'dos', 'm..."
1,New Cluster #2,36.6,26.3,4.1,36.7,85.7,36.7,1.0,"['new', 'case', 'apr', 'death', 'access', 'rep...","['new', 'case', 'active', 'apr', 'store', 'dea..."
2,New Cluster #5,-20.2,-19.6,-40.9,-37.4,-14.3,-19.7,2.0,"['got', 'second', 'pfizer', 'moderna', 'shot',...","['got', 'shot', 'second', 'moderna', 'dose', '..."
